# DOE Reference Building Analysis for Climate Region:

In [1]:
# set climate region to match weatherfile
climate_region = '5a'

### Using EnergyPlus version:

In [2]:
ep_version = '9.2'

### Import libraries and configure workbook

In [3]:
import archetypal as ar
import pandas as pd
import os
from path import Path

# to disable, %matplotlib inline
#%matplotlib notebook
%matplotlib inline

ar.config(use_cache=True)

c:\users\mariana\desktop\projects\buildingtechnology\containerfarmep\archetypal_files\archetypal\archetypal\utils.py:117: UserWarning: The TRNSYS path does not exist. Please set the TRNSYS path with the --trnsys-default-folder option
  warnings.warn(


### Requirements

This workbook uses a branched version of archetypal to be able to index idf.meters.OutputMeter[ ] objects. 

This workbook should be run in a directory that containes a subdirectory named according to the climate region (e.g., "5a"). Within the climate region directory, there need to be the following directories:
    
    IDF: contains all the DOE reference building IDF files for the climate region
    UMI: if applicable, contains any UMI results in the format of annual hourly results
    WeatherFile: contains the weather file for the climate region. 

### Global Variables

Lists of all reference buildings and Epochs to match the string names used in the IDF files. Additionally, the headers that will be used in the results dataframes match the hourly results categories outpud from UMI. 

In [4]:
# ----------------------------------------------------------
# ---------------- GLOBAL VARIABLES ------------------------
# ----------------------------------------------------------

all_buildings = ['LargeOffice',
                 'MediumOffice',
                 'SmallOffice',
                 'Warehouse',
                 'Stand-aloneRetail',
                 'StripMall',
                 'PrimarySchool',
                 'SecondarySchool',
                 'Supermarket',
                 'QuickServiceRestaurant',
                 'FullServiceRestaurant',
                 'Hospital',
                 'OutPatient',
                 'SmallHotel',
                 'LargeHotel',
                 'MidriseApartment']
all_epochs = ["Pre1980", "Post1980", "New2004"]

energy_header = ['Cooling (kWh/m2)',
                  'Heating (kWh/m2)',
                  'Lights (kWh/m2)',
                  'Equipment (kWh/m2)',
                  'DHW (kWh/m2)',
                  'EUI (kWh/m2)']
sim_header = ['EP','UMI','error']

date_index = pd.date_range(start='2018/01/01', end='2019/1/1', closed='left', freq='H')

### Function to Generate Results Dataframe for a given DOE IDF File

This function takes in an IDF file and calculates the normalized energy consumed for cooling, heating, lighting, equipment, and hot water end uses. Additionally, this function calculates a total EUI in kWh/m2 as the sum of these five energy use categories. 

The function returns a dataframe with these values for 8760 hours. 

In [5]:
def getResultsDF(idf):
    #conditioned floor area used to normalize EUI results
    area = idf.net_conditioned_building_area #m2
    
    #local dateindex values
    date_index_local = pd.date_range(start='2018/01/01', end='2019/1/1', closed='left', freq='H')
    
    #create a blank dataframe to store results
    results = pd.DataFrame(index=date_index_local)

    # cooling end uses
    cooling_energy = (
        "CoolingCoils__EnergyTransfer"
    )
    # heating end uses
    heating_energy = (
        "HeatingCoils__EnergyTransfer",
        "Baseboard__EnergyTransfer"
    )
    
    # get total cooling and total heating energy
    total_cooling_energy = pd.Series(0,index=date_index_local)
    for item in cooling_energy:
        try:
            total_cooling_energy += (idf.meters.OutputMeter[item].values("kWh"))
        except KeyError:
            pass  # pass if meter does not exist for model
    
    total_heating_energy = pd.Series(0,index=date_index_local)
    for item in heating_energy:
        try:
            total_heating_energy += (idf.meters.OutputMeter[item].values("kWh"))
        except KeyError:
            pass  # pass if meter does not exist for model
    
    # calculate ratio of cooling and heating energy for use in cooling and heating fan and pump energy
    ratio_cooling = total_cooling_energy / (total_cooling_energy+total_heating_energy)
    ratio_heating = total_heating_energy / (total_cooling_energy+total_heating_energy)

      
    #calcualte fans electricity for cooling and heating

    if 'Fans__Electricity' in idf.meters.OutputMeter.__dict__:
        fans_cooling = idf.meters.OutputMeter.Fans__Electricity.values('kWh') * ratio_cooling /area
        fans_heating = idf.meters.OutputMeter.Fans__Electricity.values('kWh') * ratio_heating /area       
    else:
        fans_cooling = pd.Series(0,index=date_index_local)
        fans_heating = pd.Series(0,index=date_index_local)
    
   
    #calculate pumps electricity for cooling and heating
    
    if 'Pumps__Electricity' in idf.meters.OutputMeter.__dict__:
        pumps_cooling = idf.meters.OutputMeter.Pumps__Electricity.values('kWh') * ratio_cooling /area
        pumps_heating = idf.meters.OutputMeter.Pumps__Electricity.values('kWh') * ratio_heating /area
    else:
        pumps_cooling = pd.Series(0,index=date_index_local)
        pumps_heating = pd.Series(0,index=date_index_local)
    
    #cooling
    if 'Cooling__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Cooling (kWh/m2)'] = idf.meters.OutputMeter.Cooling__Electricity.values(units = ('kWh'))/area
    else:
        results['Cooling (kWh/m2)'] = pd.Series(0,index=date_index_local)
        
    if 'HeatRejection__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Cooling (kWh/m2)'] += idf.meters.OutputMeter.HeatRejection__Electricity.values(units = ('kWh'))/area

    if 'Cooling__Gas' in idf.meters.OutputMeter.__dict__:
        results['Cooling (kWh/m2)'] += idf.meters.OutputMeter.Cooling__Gas.values(units = ('kWh'))/area
  
    if 'Cooling__DistrictCooling' in idf.meters.OutputMeter.__dict__:
        results['Cooling (kWh/m2)'] += idf.meters.OutputMeter.Cooling__DistrictCooling.values(units = ('kWh'))/area

    if 'Fans__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Cooling (kWh/m2)'] += fans_cooling
        
    if 'Pumps__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Cooling (kWh/m2)'] += pumps_cooling

    
    #heating
    if 'Heating__Gas' in idf.meters.OutputMeter.__dict__:
        results['Heating (kWh/m2)'] = idf.meters.OutputMeter.Heating__Gas.values(units = ('kWh'))/area 
    else:
        results['Heating (kWh/m2)'] = pd.Series(0,index=date_index_local)
        
    if 'Heating__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Heating (kWh/m2)'] += idf.meters.OutputMeter.Heating__Electricity.values(units = ('kWh'))/area
        
    if 'Heating__DistrictHeating' in idf.meters.OutputMeter.__dict__:
        results['Heating (kWh/m2)'] += idf.meters.OutputMeter.Heating__DistrictHeating.values(units = ('kWh'))/area
        
    if 'Heating__Oil' in idf.meters.OutputMeter.__dict__:
        results['Heating (kWh/m2)'] += idf.meters.OutputMeter.Heating__Oil.values(units = ('kWh'))/area 
        
    if 'Fans__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Cooling (kWh/m2)'] += fans_heating
        
    if 'Pumps__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Cooling (kWh/m2)'] += pumps_heating

    
    #lights
    if 'InteriorLights__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Lights (kWh/m2)'] = idf.meters.OutputMeter.InteriorLights__Electricity.values(units = ('kWh'))/area
    else:
        results['Lights (kWh/m2)'] = pd.Series(0,index=date_index_local)
        
    
    #equipment
    if 'InteriorEquipment__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Equipment (kWh/m2)'] = idf.meters.OutputMeter.InteriorEquipment__Electricity.values(units = ('kWh'))/area
    else:
        results['Equipment (kWh/m2)'] = pd.Series(0,index=date_index_local)
        
    if 'Refrigeration__Electricity' in idf.meters.OutputMeter.__dict__:
        results['Equipment (kWh/m2)'] += idf.meters.OutputMeter.Refrigeration__Electricity.values(units = ('kWh'))/area
        
    
    #hot water 
    if 'WaterSystems__Gas' in idf.meters.OutputMeter.__dict__:
        results['DHW (kWh/m2)'] = idf.meters.OutputMeter.WaterSystems__Gas.values(units = ('kWh'))/area
    else:
        results['DHW (kWh/m2)'] = pd.Series(0,index=date_index_local)
        
    if 'WaterSystems__Electricity' in idf.meters.OutputMeter.__dict__:
        results['DHW (kWh/m2)'] += idf.meters.OutputMeter.WaterSystems__Electricity.values(units = ('kWh'))/area

    
    #total EUI
    results['EUI (kWh/m2)'] = results.apply(lambda row: 
                                              row['DHW (kWh/m2)']+
                                              row['Equipment (kWh/m2)']+
                                              row['Lights (kWh/m2)']+
                                              row['Heating (kWh/m2)']+
                                              row['Cooling (kWh/m2)'], 
                                              axis = 1)
    
    return results

### Run this cell to get the weather file

In [14]:
# get weather file
ep_weather = next(iter((Path(climate_region) / 'WeatherFile').files("*.epw")), None)
if not ep_weather:
    raise Exception("No weather file found")  

### Control Variables for Scripts Below

If all EnergyPlus **simulations have already been completed** for all IDF files in the climate region, `simulate = False`. Otherwise, set `simulate = True`.

If you are running scripts in **testing mode** (e.g., you don't want to run through all buildings or you don't want to overwrite any results), set `testing = True` and specify which buildings you want to use in your test script. 

If there are **no UMI results** but still want to generate a worksheet with space for UMI results, set `compare_umi = False`.

In [19]:
simulate=False

testing = True
all_buildings_test = ['Warehouse',
                      'Stand-aloneRetail']

compare_umi=False

Blank dictionary to store IDFs. Only run once for each notebook (then set `run = False`)

In [ ]:
run = False

if run:
    idfs = {}

### Script to generate an Excel spreadsheet with EnergyPlus results only (*no UMI comparison*)

If an EP-Results.xlsx file already exists, every time the script is run, the results will be appended as a new sheet. If you want to start over creating an EP-Results.xlsx file, delete the existing one. 

In [8]:
run1 = True

if run1:
     # create code for file
    if testing:
        file_code = '_test_' 
        buildings_list = all_buildings_test
    else:
        file_code = '_'
        buildings_list = all_buildings


    #check if file already exists
    fname = Path(climate_region) / f'{climate_region}{file_code}EP-Results.xlsx'
    
    if fname.isfile():
        w_a = 'a'
    else:
        w_a = 'w'

    #excel document to save results (mode='w' for write, mode='a' for append)
    with pd.ExcelWriter(fname, mode=w_a) as writer:

        #loop through all building types
        for bldg in buildings_list: 

            #empty list of results dataframes for the epoch
            epoch_dfs = []

            # loop through all epochs
            for epoch in all_epochs:
                ep_file = next(iter((Path(climate_region) / "IDF").files(f"*{bldg}{epoch}*.idf")), None)
                if not ep_file:
                    raise FileNotFoundError(f"Cound not find file for {bldg}{epoch}")

                #read idf
                idfs[f"{bldg}{epoch}"] = ar.IDF(ep_file, ep_weather, as_version = ep_version)

                if simulate:
                    #simulate idf
                    idfs[f"{bldg}{epoch}"].simulate()

                #get results and make dataframe for epoch + building combo
                results = getResultsDF(idfs[f"{bldg}{epoch}"])

                #group results by monthly and add totals row
                monthly_results = results.groupby(results.index.month).sum()
                monthly_results.index.map(str)

                # add total row
                monthly_w_tot = monthly_results.append(monthly_results.sum().rename('Annual'))

                #concat results with umi and error dataframes
                #results_all 

                #swap the order of the headers and sort
                #results_swapped = results_all.swaplevel(axis=1).sort_index(axis=1).loc[:, (energy_header, sim_header)]

                #add temporary results to list of epoch results
                epoch_dfs.append(monthly_w_tot)

            # concat all dfs
            annual = pd.concat(epoch_dfs, keys=all_epochs, names = ['Epoch','Energy Type'], axis = 1)
            annual.index.name = 'Month'

            # get floor area
            area_blg = idfs[f"{bldg}{epoch}"].net_conditioned_building_area
            # add area to sheet name
            Name_Area = f"{bldg}_{round(area_blg)}m2"

            # write results of building to new sheet
            annual.to_excel(writer, sheet_name=Name_Area)


### Script to read in EP-Results.xlsx file into a DataFrames

The script reads in the Excel file and stores each building Sheet as a dataframe.

In [10]:
ls

 Volume in drive C is Windows
 Volume Serial Number is B253-0D67

 Directory of C:\Users\Mariana\Desktop\Projects\BuildingTechnology\DOE_Eplus_RefBuildings\DOE_Analysis\5a

01/06/2021  03:44 PM    <DIR>          .
01/06/2021  03:44 PM    <DIR>          ..
01/06/2021  03:39 PM            71,117 5a_EP-Results.xlsx
01/06/2021  03:44 PM            11,036 5a_test_EP-Results.xlsx
01/06/2021  11:56 AM    <DIR>          cache
12/22/2020  05:46 PM    <DIR>          data
12/22/2020  05:43 PM    <DIR>          IDF
12/22/2020  05:46 PM    <DIR>          images
12/22/2020  05:46 PM    <DIR>          logs
01/06/2021  03:25 PM    <DIR>          UMI
12/22/2020  05:43 PM    <DIR>          WeatherFile
               2 File(s)         82,153 bytes
               9 Dir(s)  79,335,895,040 bytes free


In [12]:
ls

 Volume in drive C is Windows
 Volume Serial Number is B253-0D67

 Directory of C:\Users\Mariana\Desktop\Projects\BuildingTechnology\DOE_Eplus_RefBuildings\DOE_Analysis

01/06/2021  05:38 PM    <DIR>          .
01/06/2021  05:38 PM    <DIR>          ..
12/22/2020  05:42 PM             1,928 .gitignore
12/22/2020  05:47 PM    <DIR>          .ipynb_checkpoints
01/06/2021  03:44 PM    <DIR>          5a
01/06/2021  05:34 PM    <DIR>          cache
01/06/2021  05:34 PM    <DIR>          data
01/06/2021  05:38 PM            44,756 DOE_Analysys.ipynb
01/06/2021  05:34 PM    <DIR>          images
01/06/2021  05:34 PM    <DIR>          logs
12/22/2020  05:42 PM                62 README.md
               3 File(s)         46,746 bytes
               8 Dir(s)  79,338,045,440 bytes free


In [13]:


results_fname = f'{climate_region}/{climate_region}_EP-Results.xlsx'

pd.read_excel(results_fname)




XLRDError: Excel xlsx file; not supported

### Script to pull in UMI results and Compare with EnergyPlus Results

If EnergyPlus results already exist in an EP-Results.xlsx file, this script reads those in. Otherwise, the script runs EnergyPlus simulations and creates two files: EP-Results.xlsx and ResultsComparison.xlsx.

In [46]:
run2 = False

if run2:

    os.chdir(climate_region)

    # create blank dictionary to store IDFs
    idfs = {}

    # create code for file
    if testing:
        file_code = '_test_' 
        buildings_list = all_buildings_test
    else:
        file_code = '_'
        buildings_list = all_buildings


    #check if file already exists
    fname = f'{climate_region}{file_code}EP-Results.xlsx'
    if os.path.isfile(fname):
        w_a = 'a'
    else:
        w_a = 'w'

    #excel document to save results (mode='w' for write, mode='a' for append)
    with pd.ExcelWriter(fname, mode=w_a) as writer:

        #loop through all building types
        for bldg in buildings_list: 

            #empty list of results dataframes for the epoch
            epoch_dfs = []

            # loop through all epochs
            for epoch in all_epochs:
                ep_file = next(iter(Path("IDF").files(f"*{bldg}{epoch}*.idf")), None)
                if not ep_file:
                    raise FileNotFoundError(f"Cound not find file for {bldg}{epoch}")

                #read idf
                idfs[f"{bldg}{epoch}"] = ar.IDF(ep_file, ep_weather, as_version = ep_version)

                if simulate:
                    #simulate idf
                    idfs[f"{bldg}{epoch}"].simulate()

                #get results and make dataframe for epoch + building combo
                results = getResultsDF(idfs[f"{bldg}{epoch}"])

                #group results by monthly and att totals row
                monthly_results = results.groupby(results.index.month).sum()
                monthly_results.index.map(str)

                # add total row
                monthly_tot = monthly_results.append(monthly_results.sum().rename('Annual'))

                # read in umi file
                if compare_umi:
                    py_umi = pd.DataFrame(0,index=date_index,columns=energy_header)  
                    #####################################################
                    ###### SAM TO ADD CODE HERE ######
                    # this will need to be replaced with a function that reads 
                    # results from pyumi for each building-epoch combo
                    #####################################################
                else:
                    py_umi = pd.DataFrame(0,index=date_index,columns=energy_header)  

                #reformat umi results
                umi_month = py_umi.groupby(py_umi.index.month).sum()
                umi_month.index.map(str)
                umi = umi_month.append(umi_month.sum().rename('Annual'))

                # create error dataframe
                error_df = (umi-monthly_tot)/monthly_tot

                #concat results with umi and error dataframes
                results_all = pd.concat([monthly_tot, umi, error_df], 
                                             keys = sim_header, 
                                             names=['Simulation Type','Energy Type'], 
                                             axis = 1)
                #swap the order of the headers and sort
                results_swapped = results_all.swaplevel(axis=1).sort_index(axis=1).loc[:, (energy_header, sim_header)]

                # add floor area row
                area_blg = idfs[f"{bldg}{epoch}"].net_conditioned_building_area
                area_df = pd.DataFrame(area_blg,index=['Conditioned Area m2'],columns=results_swapped.columns)
                results_with_area = pd.concat([results_swapped,area_df])

                #add temporary results to list of epoch results
                epoch_dfs.append(results_with_area)

            # concat all dfs
            annual = pd.concat(epoch_dfs, keys=all_epochs, names = ['Epoch','Energy Type','Simulation Type'], axis = 1)
            annual.index.name = 'Month'

            # get floor area
            area_blg = idfs[f"{bldg}{epoch}"].net_conditioned_building_area
            # add area to sheet name
            Name_Area = f"{bldg}_{round(area_blg)}m2"

            # write results of building to new sheet
            annual.to_excel(writer, sheet_name=Name_Area)

    os.chdir("..")

In [20]:
#### OLD CODE ######

run3 = False

if run3:

    # create blank dictionary to store IDFs
    idfs = {}

    # create code for file
    if testing:
        file_code = '_test_' 
        buildings_list = all_buildings_test
    else:
        file_code = '_'
        buildings_list = all_buildings


    #check if file already exists
    fname = f'{climate_region}{file_code}ResultsComparison.xlsx'
    if os.path.isfile(fname):
        w_a = 'a'
    else:
        w_a = 'w'

    #excel document to save results (mode='w' for write, mode='a' for append)
    with pd.ExcelWriter(fname, mode=w_a) as writer:

        #loop through all building types
        for bldg in buildings_list: 

            #empty list of results dataframes for the epoch
            epoch_dfs = []

            # loop through all epochs
            for epoch in all_epochs:
                ep_file = next(iter(Path("IDF").files(f"*{bldg}{epoch}*.idf")), None)
                if not ep_file:
                    raise FileNotFoundError(f"Cound not find file for {bldg}{epoch}")

                #read idf
                idfs[f"{bldg}{epoch}"] = ar.IDF(ep_file, ep_weather, as_version = ep_version)

                if simulate:
                    #simulate idf
                    idfs[f"{bldg}{epoch}"].simulate()

                #get results and make dataframe for epoch + building combo
                results = getResultsDF(idfs[f"{bldg}{epoch}"])

                #group results by monthly and att totals row
                monthly_results = results.groupby(results.index.month).sum()
                monthly_results.index.map(str)

                # add total row
                monthly_tot = monthly_results.append(monthly_results.sum().rename('Annual'))

                # read in umi file
                if compare_umi:
                    py_umi = pd.DataFrame(0,index=date_index,columns=energy_header)  
                    #####################################################
                    ###### SAM TO ADD CODE HERE ######
                    # this will need to be replaced with a function that reads 
                    # results from pyumi for each building-epoch combo
                    #####################################################
                else:
                    py_umi = pd.DataFrame(0,index=date_index,columns=energy_header)  

                #reformat umi results
                umi_month = py_umi.groupby(py_umi.index.month).sum()
                umi_month.index.map(str)
                umi = umi_month.append(umi_month.sum().rename('Annual'))

                # create error dataframe
                error_df = (umi-monthly_tot)/monthly_tot

                #concat results with umi and error dataframes
                results_all = pd.concat([monthly_tot, umi, error_df], 
                                             keys = sim_header, 
                                             names=['Simulation Type','Energy Type'], 
                                             axis = 1)
                #swap the order of the headers and sort
                results_swapped = results_all.swaplevel(axis=1).sort_index(axis=1).loc[:, (energy_header, sim_header)]

                # add floor area row
                area_blg = idfs[f"{bldg}{epoch}"].net_conditioned_building_area
                area_df = pd.DataFrame(area_blg,index=['Conditioned Area m2'],columns=results_swapped.columns)
                results_with_area = pd.concat([results_swapped,area_df])

                #add temporary results to list of epoch results
                epoch_dfs.append(results_with_area)

            # concat all dfs
            annual = pd.concat(epoch_dfs, keys=all_epochs, names = ['Epoch','Energy Type','Simulation Type'], axis = 1)
            annual.index.name = 'Month'

            # get floor area
            area_blg = idfs[f"{bldg}{epoch}"].net_conditioned_building_area
            # add area to sheet name
            Name_Area = f"{bldg}_{round(area_blg)}m2"

            # write results of building to new sheet
            annual.to_excel(writer, sheet_name=Name_Area)

ExpandObjects #0-RefBldgOutPatientPre1980_v1.4_7.2_5A_USA_IL_CHICAGO-OHARE.idf: 3.00it [00:00, 3.35it/s]
EnergyPlus #0-RefBldgOutPatientPre1980_v1.4_7.2_5A_USA_IL_CHICAGO-OHARE.idf: 210it [09:27, 2.70s/it] 
ExpandObjects #0-RefBldgOutPatientPost1980_v1.4_7.2_5A_USA_IL_CHICAGO-OHARE.idf: 3.00it [00:00, 3.74it/s]
EnergyPlus #0-RefBldgOutPatientPost1980_v1.4_7.2_5A_USA_IL_CHICAGO-OHARE.idf: 211it [08:19, 2.37s/it] 
ExpandObjects #0-RefBldgOutPatientNew2004_v1.4_7.2_5A_USA_IL_CHICAGO-OHARE.idf: 3.00it [00:00, 3.52it/s]
EnergyPlus #0-RefBldgOutPatientNew2004_v1.4_7.2_5A_USA_IL_CHICAGO-OHARE.idf: 213it [12:40, 3.57s/it] 
ExpandObjects #0-RefBldgSmallHotelPre1980_v1.4_7.2_5A_USA_IL_CHICAGO-OHARE.idf: 3.00it [00:01, 2.33it/s]
EnergyPlus #0-RefBldgSmallHotelPre1980_v1.4_7.2_5A_USA_IL_CHICAGO-OHARE.idf: 179it [07:35, 2.55s/it] 
ExpandObjects #0-RefBldgSmallHotelPost1980_v1.4_7.2_5A_USA_IL_CHICAGO-OHARE.idf: 3.00it [00:00, 6.15it/s]
EnergyPlus #0-RefBldgSmallHotelPost1980_v1.4_7.2_5A_USA_IL_CHICA

Quick umi compare (with a completed DOE results excel file